# Transfere Learning with VGG16

+ [6 Open Source Data Science Projects to Impress your Interviewer](https://www.analyticsvidhya.com/blog/2020/06/6-open-source-data-science-projects-interviewer/)
+ [A Newbie-Friendly Guide to Transfer Learning](https://www.v7labs.com/blog/transfer-learning-guide)
+ [Transfer Learning with VGG16 and Keras](https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4)

## 1. Get Data

@ONLINE {tfflowers,
author = "The TensorFlow Team",
title = "Flowers",
month = "jan",
year = "2019",
url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

### 1.1. Load Data

In [2]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


### 1.2. Prepare data

In [3]:
## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

## 2.  Obtain pre-trained model

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)


58900480/58889256 [==============================] - 0s 0us/step


In [5]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

## 3. Create a base model

In [6]:
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

## 4. Freeze layers

## 5. Add new trainable layers

In [7]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

## 6. Train the new layers

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])


Epoch 1/50
65/65 [==============================] - 29s 254ms/step - loss: 1.8615 - accuracy: 0.3280 - val_loss: 1.3557 - val_accuracy: 0.3735
Epoch 2/50
65/65 [==============================] - 12s 187ms/step - loss: 1.1739 - accuracy: 0.5085 - val_loss: 1.1993 - val_accuracy: 0.6109
Epoch 3/50
65/65 [==============================] - 12s 188ms/step - loss: 0.9258 - accuracy: 0.6852 - val_loss: 1.0992 - val_accuracy: 0.6498
Epoch 4/50
65/65 [==============================] - 12s 190ms/step - loss: 0.6940 - accuracy: 0.7674 - val_loss: 1.1653 - val_accuracy: 0.6615
Epoch 5/50
65/65 [==============================] - 12s 187ms/step - loss: 0.5376 - accuracy: 0.8156 - val_loss: 0.9804 - val_accuracy: 0.6770
Epoch 6/50
65/65 [==============================] - 12s 188ms/step - loss: 0.4504 - accuracy: 0.8384 - val_loss: 0.9479 - val_accuracy: 0.7082
Epoch 7/50
65/65 [==============================] - 12s 190ms/step - loss: 0.3163 - accuracy: 0.8803 - val_loss: 0.9683 - val_accuracy: 0.7082

## 7. 6. Fine-tune your model

# Extra: comparing to hand-made model

In [9]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling


hand_made_model = Sequential()
hand_made_model.add(Rescaling(1./255, input_shape=(150,150,3)))

hand_made_model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
hand_made_model.add(layers.MaxPooling2D(3))

hand_made_model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
hand_made_model.add(layers.MaxPooling2D(2))

hand_made_model.add(layers.Flatten())
hand_made_model.add(layers.Dense(50, activation='relu'))
hand_made_model.add(layers.Dense(20, activation='relu'))
hand_made_model.add(layers.Dense(5, activation='softmax'))


hand_made_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

hand_made_model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 5s 48ms/step - loss: 1.5291 - accuracy: 0.2876 - val_loss: 1.4355 - val_accuracy: 0.3424
Epoch 2/50
65/65 [==============================] - 3s 40ms/step - loss: 1.3618 - accuracy: 0.3776 - val_loss: 1.3097 - val_accuracy: 0.3969
Epoch 3/50
65/65 [==============================] - 3s 41ms/step - loss: 1.2744 - accuracy: 0.4360 - val_loss: 1.2434 - val_accuracy: 0.4747
Epoch 4/50
65/65 [==============================] - 3s 40ms/step - loss: 1.2159 - accuracy: 0.4574 - val_loss: 1.2038 - val_accuracy: 0.4553
Epoch 5/50
65/65 [==============================] - 3s 41ms/step - loss: 1.1725 - accuracy: 0.4861 - val_loss: 1.1871 - val_accuracy: 0.5350
Epoch 6/50
65/65 [==============================] - 3s 41ms/step - loss: 1.0920 - accuracy: 0.5328 - val_loss: 1.1951 - val_accuracy: 0.5156
Epoch 7/50
65/65 [==============================] - 3s 41ms/step - loss: 1.0067 - accuracy: 0.5878 - val_loss: 1.2228 - val_accuracy: 0.5253
Epoch 8/50
65